In [40]:
import json
import base64
import qrcode
import requests
import datetime
import uuid
import matplotlib.pyplot as plt
from pymongo import MongoClient
from typing import Optional, List
from didcomm.common.types import DID, VerificationMethodType, VerificationMaterial, VerificationMaterialFormat
from didcomm.did_doc.did_doc import DIDDoc, VerificationMethod, DIDCommService
from didcomm.did_doc.did_resolver import DIDResolver
from didcomm.message import Message, FromPrior
from didcomm.secrets.secrets_resolver_demo import SecretsResolverDemo
from didcomm.unpack import unpack, UnpackResult
from didcomm.message import Attachment, AttachmentDataJson, AttachmentDataLinks
from didcomm.common.resolvers import ResolversConfig
from didcomm.pack_encrypted import pack_encrypted, PackEncryptedConfig, PackEncryptedResult
from peerdid.core.did_doc_types import DIDCommServicePeerDID
from didcomm.secrets.secrets_util import generate_x25519_keys_as_jwk_dict, generate_ed25519_keys_as_jwk_dict, jwk_to_secret
from peerdid import peer_did
from peerdid.did_doc import DIDDocPeerDID
from peerdid.types import VerificationMaterialAuthentication, VerificationMethodTypeAuthentication, VerificationMaterialAgreement, VerificationMethodTypeAgreement, VerificationMaterialFormatPeerDID

In [41]:
secrets_resolver = SecretsResolverDemo()

In [42]:
class DIDResolverPeerDID(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        did_doc_json = peer_did.resolve_peer_did(did, format = VerificationMaterialFormatPeerDID.JWK)
        did_doc = DIDDocPeerDID.from_json(did_doc_json)

        return DIDDoc(
            did=did_doc.did,
            key_agreement_kids = did_doc.agreement_kids,
            authentication_kids = did_doc.auth_kids,
            verification_methods = [
                VerificationMethod(
                    id = m.id,
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m.controller,
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m.ver_material.value)
                    )
                )
                for m in did_doc.authentication + did_doc.key_agreement
            ],
            didcomm_services = [
                DIDCommService(
                    id = s.id,
                    service_endpoint = s.service_endpoint[0]["uri"] if "uri" in s.service_endpoint[0] else s.service_endpoint[0],
                    routing_keys = s.routing_keys,
                    accept = s.accept
                )
                for s in did_doc.service
                if isinstance(s, DIDCommServicePeerDID)
            ] if did_doc.service else []
        )

In [43]:
async def create_peer_did(self,
                        auth_keys_count: int = 1,
                        agreement_keys_count: int = 1,
                        service_endpoint: Optional[str] = None,
                        service_routing_keys: Optional[List[str]] = None
                        ) -> str:
        # 1. generate keys in JWK format
        agreem_keys = [generate_x25519_keys_as_jwk_dict() for _ in range(agreement_keys_count)]
        auth_keys = [generate_ed25519_keys_as_jwk_dict() for _ in range(auth_keys_count)]

        # 2. prepare the keys for peer DID lib
        agreem_keys_peer_did = [
            VerificationMaterialAgreement(
                type=VerificationMethodTypeAgreement.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in agreem_keys
        ]
        auth_keys_peer_did = [
            VerificationMaterialAuthentication(
                type=VerificationMethodTypeAuthentication.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in auth_keys
        ]

        # 3. generate service
        service = None
        if service_endpoint:
            service = json.dumps(
                DIDCommServicePeerDID(
                    id="new-id",
                    service_endpoint=service_endpoint, 
                    routing_keys=service_routing_keys,
                    accept=["didcomm/v2"]
                ).to_dict()
            )

        # 4. call peer DID lib
        # if we have just one key (auth), then use numalg0 algorithm
        # otherwise use numalg2 algorithm
        if len(auth_keys_peer_did) == 1 and not agreem_keys_peer_did and not service:
            did = peer_did.create_peer_did_numalgo_0(auth_keys_peer_did[0])
        else:
            did = peer_did.create_peer_did_numalgo_2(
                encryption_keys=agreem_keys_peer_did,
                signing_keys=auth_keys_peer_did,
                service=service,
            )

        # 5. set KIDs as in DID DOC for secrets and store the secret in the secrets resolver
        did_doc = DIDDocPeerDID.from_json(peer_did.resolve_peer_did(did))
        for auth_key, kid in zip(auth_keys, did_doc.auth_kids):
            private_key = auth_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        for agreem_key, kid in zip(agreem_keys, did_doc.agreement_kids):
            private_key = agreem_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        return did


### Get OOB get somehow Alice DID (not covered here)

In [44]:
alice_did = "did:peer:2.Ez6LSp4iKH9cf4dBrktYkwVMpLyfhVwyFNWYSBwvMbQBcJcMn.Vz6MkvHVxC2yQLALNfmZhvbniC96yzYBC3CQtvaXmQWC22C3c.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOlt7InVyaSI6ImRpZDpwZWVyOjIuRXo2TFNqM1hXZ1p6YmdOeHY4aFVuRHB5NXFraGVNcVBucEtpRFZ3a0ZrQUhBd000Zi5WejZNa2VaSDNXejdHZHRtd0ROY25hMzdVQlFNV1VWR1NZNG5XWVI1aGlidlF5RXNtLlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd2N6b3ZMMjFsWkdsaGRHOXlMbkp2YjNSemFXUXVZMnh2ZFdRaUxDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCJ9XSwiYSI6WyJkaWRjb21tL3YyIl19"

### Get mediator DID from Alice DID and endpoint from mediator DID

In [45]:
alice_did_doc = json.loads(peer_did.resolve_peer_did(alice_did))
mediator_did = alice_did_doc["service"][0]["serviceEndpoint"][0]["uri"]
print(mediator_did)

did:peer:2.Ez6LSj3XWgZzbgNxv8hUnDpy5qkheMqPnpKiDVwkFkAHAwM4f.Vz6MkeZH3Wz7GdtmwDNcna37UBQMWUVGSY4nWYR5hibvQyEsm.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0


In [46]:
mediator_did_doc = json.loads(peer_did.resolve_peer_did(mediator_did))
mediator_endpoint = mediator_did_doc["service"][0]["serviceEndpoint"]
print(mediator_endpoint)

https://mediator.rootsid.cloud


### BOB creates DID and a basic msg to Bob

In [47]:
bob_did_to_alice = await create_peer_did(1,1, service_endpoint="https://www.example.com/bob")
print("Bob's DID:", bob_did_to_alice)

Bob's DID: did:peer:2.Ez6LSqwg44tAhSiEsXzyso44Xsvdbw4sZBdADL1p5cEP67u6Z.Vz6MkvtTM54f2v2He2ZCeCeukHLZbdVTwSr5vzL3TxGhjAqwD.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL3d3dy5leGFtcGxlLmNvbS9ib2IiLCJhIjpbImRpZGNvbW0vdjIiXX0


In [48]:

bob_basic_message = Message(
    id = str(uuid.uuid4()),
    type="https://didcomm.org/basicmessage/2.0/message",
    body={"content": "Hola Alice"},
    created_time= int(datetime.datetime.now().timestamp())            
)

In [49]:
bob_basic_message_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = bob_basic_message,
    frm = bob_did_to_alice,
    to = alice_did,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

In [50]:
print(json.loads(bob_basic_message_packed.packed_msg)["protected"])

eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbkJsWlhJNk1pNUZlalpNVTNGM1p6UTBkRUZvVTJsRmMxaDZlWE52TkRSWWMzWmtZbmMwYzFwQ1pFRkVUREZ3TldORlVEWTNkVFphTGxaNk5rMXJkblJVVFRVMFpqSjJNa2hsTWxwRFpVTmxkV3RJVEZwaVpGWlVkMU55TlhaNlRETlVlRWRvYWtGeGQwUXVVMlY1U25CYVEwazJTVzAxYkdSNU1YQmFRMGx6U1c1UmFVOXBTbXRpVTBselNXNU5hVTlwU205a1NGSjNZM3B2ZGt3elpETmtlVFZzWlVkR2RHTkhlR3hNYlU1MllsTTVhV0l5U1dsTVEwcG9TV3B3WWtsdFVuQmFSMDUyWWxjd2RtUnFTV2xZV0RBak5reFRjWGRuTkRSMFFXaFRhVVZ6V0hwNWMyODBORmh6ZG1SaWR6UnpXa0prUVVSTU1YQTFZMFZRTmpkMU5sbyIsImFwdiI6IkRfb1RFR1ZzYVFFLUNKaloyRFJTVEJac1VIRmEwcXA5RnI0Z1hRT0VwN1EiLCJza2lkIjoiZGlkOnBlZXI6Mi5FejZMU3F3ZzQ0dEFoU2lFc1h6eXNvNDRYc3ZkYnc0c1pCZEFETDFwNWNFUDY3dTZaLlZ6Nk1rdnRUTTU0ZjJ2MkhlMlpDZUNldWtITFpiZFZUd1NyNXZ6TDNUeEdoakFxd0QuU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3Y3pvdkwzZDNkeTVsZUdGdGNHeGxMbU52YlM5aWIySWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAjNkxTcXdnNDR0QWhTaUVzWHp5c280NFhz

### Bobs cread DID and msg routed via Mediator

In [51]:
bob_did_to_mediator = await create_peer_did(1,1, service_endpoint="https://www.example.com/bob")

In [52]:
bob_routed_message = Message(
    id = str(uuid.uuid4()),
    type="https://didcomm.org/routing/2.0/forward",
    body={"next": alice_did},
    to=[mediator_did],
    attachments=[Attachment(
            data=AttachmentDataJson(json=json.loads(bob_basic_message_packed.packed_msg))
        )]
)

In [53]:
bob_routed_message_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = bob_routed_message,
    frm = bob_did_to_mediator,
    to = mediator_did,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

### Sending to Mediator

In [54]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(mediator_endpoint, headers=headers, data = bob_routed_message_packed.packed_msg)
